In [1]:
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

from PIL import Image
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import time
import random
from math import ceil
from tqdm import tqdm
import os

import torch.utils.data as data
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F

BATCH_SIZE = 10
DEVICE = torch.device('cpu')
NUM_EPOCHS = 10


In [2]:
train_data_ori = open("./E3_data/Short_train_labels.txt","r")

train_lines = train_data_ori.readlines()

train_data_new = torch.empty([0, 3, 64, 64])
train_label = []

i = 0
for line in train_lines:
    if i%20 == 0:
        print("load train dataset\t"+str(i))
    i += 1

    line_inf = line.split('\t')
    imgpath='./E3_data/train_img/'+line_inf[0]
    img = plt.imread(imgpath).astype(np.float32)
    img = torch.from_numpy(img).reshape([1, 3, 64, 64])
    train_data_new = torch.cat((train_data_new, img), 0)
 
    label = int(line_inf[1].replace("\n", ""))
    train_label.append(label)

train_label = np.array(train_label)
train_label = torch.from_numpy(train_label)


load train dataset	0
load train dataset	20
load train dataset	40
load train dataset	60
load train dataset	80


In [3]:
val_data_ori = open("./E3_data/Short_val_labels.txt","r")
val_lines = val_data_ori.readlines()

val_data_new = torch.empty([0, 3, 64, 64])
val_label = []

i = 0
for line in val_lines:
    if i%20 == 0:
        print("load val dataset\t"+str(i))
    i += 1

    line_inf = line.split('\t')
    imgpath='./E3_data/val_img/'+line_inf[0]
    img = plt.imread(imgpath).astype(np.float32)
    img = torch.from_numpy(img).reshape([1, 3, 64, 64])
    val_data_new = torch.cat((val_data_new, img), 0)
 
    label = int(line_inf[1].replace("\n", ""))
    val_label.append(label)
val_label = np.array(val_label)
val_label = torch.from_numpy(val_label)


load val dataset	0
load val dataset	20
load val dataset	40
load val dataset	60
load val dataset	80


In [4]:
test_data_ori = open("./E3_data/Short_Test_labels.txt","r")
test_lines = test_data_ori.readlines()

test_data_new = torch.empty([0, 3, 64, 64])
test_label = []

i = 0
for line in test_lines:
    if i%20 == 0:
        print("load test dataset\t"+str(i))
    i += 1

    line_inf = line.split('\t')
    imgpath='./E3_data/test_img/'+line_inf[0]
    img = plt.imread(imgpath).astype(np.float32)
    img = torch.from_numpy(img).reshape([1, 3, 64, 64])
    test_data_new = torch.cat((test_data_new, img), 0)
 
    label = int(line_inf[1].replace("\n", ""))
    test_label.append(label)
test_label = np.array(test_label)
test_label = torch.from_numpy(test_label)
test_label



load test dataset	0
load test dataset	20
load test dataset	40
load test dataset	60
load test dataset	80


tensor([ 6,  7,  1,  2,  0,  0,  6,  8,  0,  6,  4,  0,  3, 11,  2,  8,  1,  3,
         8,  3, 10,  8,  5,  5,  6,  0,  1,  0,  1,  3,  7,  7, 11,  0,  1,  0,
         8,  0,  1,  0, 11,  6,  1,  1,  2, 10, 11,  3,  8,  8,  2,  5,  9,  8,
         3,  0,  3,  5,  7,  0,  6,  0,  6,  3,  5,  0,  4,  3,  7, 11,  8,  7,
         1,  3,  7,  5,  6,  7,  3,  6,  3,  2,  5,  8,  9,  5,  5,  0,  3,  7,
        10,  1,  7, 10,  5, 10,  4,  0,  0,  4])

In [5]:
train_dataset = data.TensorDataset(train_data_new, train_label)
val_dataset = data.TensorDataset(val_data_new, val_label)
test_dataset = data.TensorDataset(test_data_new, test_label)

train_loader = data.DataLoader(
    dataset=train_dataset,      # torch TensorDataset format
    batch_size=BATCH_SIZE,      # mini batch size
    shuffle=True,               # random shuffle for training
    num_workers=4,              # subprocesses for loading data
)

val_loader = data.DataLoader(
    dataset=val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)

test_loader = data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)

In [6]:
import math

import torch.nn as nn
import torch.utils.model_zoo as model_zoo

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class Normalize(nn.Module):

    def __init__(self, power=2):
        super(Normalize, self).__init__()
        self.power = power

    def forward(self, x):
        norm = x.pow(self.power).sum(1, keepdim=True).pow(1. / self.power)
        out = x.div(norm)
        return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, low_dim=128, in_channel=3, width=1):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channel, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.base = int(64 * width)

        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, self.base, layers[0])
        self.layer2 = self._make_layer(block, self.base * 2, layers[1], stride=2)
        self.layer3 = self._make_layer(block, self.base * 4, layers[2], stride=2)
        self.layer4 = self._make_layer(block, self.base * 8, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7, stride=1)
        self.fc = nn.Linear(self.base * 8 * block.expansion, low_dim)
        self.l2norm = Normalize(2)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = [block(self.inplanes, planes, stride, downsample)]
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    def features(self, x):
        x = self.conv1(x) 
        x = self.bn1(x) 
        x = self.relu(x) 
        x = self.maxpool(x) 
        x = self.layer1(x) 
        x = self.layer2(x) 
        x = self.layer3(x) 
        x = self.layer4(x) 
        return x

    def forward(self, x): 
        x = self.features(x)
        return x

def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model


def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model


def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model



In [7]:
model = resnet152().cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())


AssertionError: Torch not compiled with CUDA enabled

In [19]:
# train the model
for epoch in range(NUM_EPOCHS):
    print("epoch:", +epoch)
    model.train()
    for images, labels in tqdm(train_loader):
        images,labels = images.to(DEVICE),labels.to(DEVICE)
        optimizer.zero_grad()
        output = model(images)
        print(output.shape)
        print(labels)



        """
        lossvalue = criterion(output,labels)
        lossvalue.backward()
        optimizer.step()
    # evaluate the model
    model.eval()
    test_loss = 0
    correct_num_test = 0
    train_loss = 0
    correct_num_train = 0
    with torch.no_grad():
        for images, labels in train_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            output = model(images)
            train_loss += torch.nn.functional.nll_loss(output, labels, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct_num_train += pred.eq(labels.view_as(pred)).sum().item()
    train_loss /= len(test_loader.dataset)
    correct_rate = 100. * correct_num_train / len(train_loader.dataset)
    print('Train Accuracy: {0}/{1}({2:.4}%)'.format(correct_num_train,len(train_loader.dataset),correct_rate))

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            output = model(images)
            test_loss += torch.nn.functional.nll_loss(output, labels, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct_num_test += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    correct_rate = 100. * correct_num_test / len(test_loader.dataset)
    print('Test  Accuracy: {0}/{1}({2:.4}%)'.format(correct_num_test,len(test_loader.dataset),correct_rate))   
    """


 10%|█         | 1/10 [00:01<00:09,  1.03s/it]tensor([[[[8.5409e-01, 4.7683e-01],
          [9.8073e-01, 5.2165e-01]],

         [[0.0000e+00, 0.0000e+00],
          [1.8349e+00, 4.5399e-01]],

         [[5.5579e-01, 1.7523e+00],
          [0.0000e+00, 0.0000e+00]],

         ...,

         [[0.0000e+00, 2.0975e-01],
          [1.0849e-02, 6.8696e-01]],

         [[1.5868e-01, 7.0362e-01],
          [3.9025e-01, 2.8307e-03]],

         [[1.9071e+00, 3.0368e+00],
          [7.7979e-01, 2.3244e+00]]],


        [[[1.3534e+00, 2.8220e-01],
          [0.0000e+00, 2.9892e-01]],

         [[4.0442e-01, 3.4313e+00],
          [2.6948e+00, 1.8273e+00]],

         [[1.0554e+00, 0.0000e+00],
          [9.6024e-01, 0.0000e+00]],

         ...,

         [[6.2870e-01, 1.4682e+00],
          [8.0668e-01, 1.2488e-01]],

         [[0.0000e+00, 3.9463e-01],
          [5.7857e-01, 0.0000e+00]],

         [[8.2183e-01, 1.5158e+00],
          [2.0949e+00, 4.0663e-01]]],


        [[[0.0000e+00, 5.9031e-0

KeyboardInterrupt: 

In [9]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(3,10,kernel_size=(5,5))
        self.conv2 = nn.Conv2d(10,20,kernel_size=(3,3))
        self.linear1 = nn.Linear(20*108*108,500)
        self.linear2 = nn.Linear(500,10)
        
    def forward(self, x):
        size_in = x.size(0)
        mid = self.conv1(x)
        mid = F.relu(mid)
        mid = F.max_pool2d(mid,2,2)
        mid = self.conv2(mid)
        mid = F.relu(mid)
        mid = mid.view(size_in,-1)
        mid = self.linear1(mid)
        mid = F.relu(mid)
        mid = self.linear2(mid)
        out = F.log_softmax(mid,dim = 1)
        return out 

model = SimpleNet()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())